In [1]:
# HPC
import numpy as np
import xarray as xr
from scipy.interpolate import interp1d

# Plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Paths and stuff
import os
import sys

import re
import warnings
warnings.filterwarnings("ignore")

sys.path.append('/home/shreyas/pySICOPOLIS/src')
from pySICOPOLIS import *

In [ ]:
ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups_develop/sico_out/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka_ser.nc')

start_idx = np.where(ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka["t"].data == -11000)[0][0]
delta_ts_init = ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka["delta_ts"].data[start_idx::5]

averaged_delta_ts = delta_ts_init.copy()
for i in range(110):
    averaged_delta_ts[i] = np.mean(ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka["delta_ts"].data[start_idx + 5*i - 2 : start_idx + 5*i + 3])
averaged_delta_ts[-1] = 0.2*(ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka["delta_ts"].data[-1]*2 + np.sum(ds_ser_grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka["delta_ts"].data[-3:]))

ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init = xr.open_dataset('/home/shreyas/update_to_develop_sicopolis/sicopolis_spinups/sico_out/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka/grl40_bm5_paleo17a_CT4_BH0_8point5CS_spinup_0ka0006.nc')

H_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/bm5_data_40kms.nc")
age_data_40 = xr.open_dataset("/scratch2/shreyas/GrIS_paleo_data/age_data_40kms.nc")

H_data = H_data_40["H"].data
H_uncert_data = H_data_40["H_uncert"].data
V_uncert_dummy2d_data = H_data_40["V_uncert_dummy2d"].data
zl_uncert_data = H_data_40["zl_uncert"].data

age_c_data = age_data_40["age_c"].data
age_c_uncert_data = age_data_40["age_c_uncert"].data

mask_age_c = np.zeros(age_c_data.shape)
for kc in range(age_c_data.shape[0]):
    for j in range(age_c_data.shape[1]):
        for i in range(age_c_data.shape[2]):
            if age_c_uncert_data[kc, j, i] > 0 and age_c_data[kc, j, i] >= 0 and age_c_data[kc, j, i] <= 134000 and H_data[j, i] >= 2000.0:
                mask_age_c[kc, j, i] = 1.0

sicopolis_dir = '/home/shreyas/update_to_develop_sicopolis/sicopolis_optim_AB_nodeltats_noconvinit_constrained_prior_stats'
simulation = 'grl40_bm5_paleo17a_CT4_BH0_AC_BM5_m11ka_pkp'

dict_sico_out_folder_prefixes = {"nodiff": "N",
                                 "tlm": "FORWARD",
                                 "adj": "ADJOINT",
                                 "tlm_action": "FORWARDHESSACTION",
                                 "adj_action": "ADJHESSACTION"}
dict_ad_exec_cmds_suffixes = {"nodiff": "nodiff",
                              "tlm": "forward",
                              "adj": "adjoint",
                              "tlm_action": "forwardhessaction",
                              "adj_action": "adjointhessaction"}
dict_ad_log_file_suffixes = {"nodiff": "nodiff",
                             "tlm": "tlm",
                             "adj": "adj",
                             "tlm_action": "tlm_hessaction",
                             "adj_action": "adj_hessaction"}
dict_ad_nc_suffixes = {"nodiff": "nodiff",
                       "tlm": "tlm",
                       "adj": "adj",
                       "tlm_action": "tlm_hessaction",
                       "adj_action": "adj_hessaction"}

KCMAX = 80
exp_sigma_level = dataCleaner.exp_sigma_level(zeta = np.arange(0,1+1./KCMAX,1./KCMAX),
                                              exponent = 2.0)
KRMAX = 40
zeta_r = np.arange(0.,1. + 1.0/KRMAX, 1.0/KRMAX)
xModel40       = np.arange(-72.,97.,4.0)*10
yModel40       = np.arange(-345.,-56.,4.0)*10
time_ad = np.arange(111, dtype=float)
IMAX = xModel40.shape[0]-1
JMAX = yModel40.shape[0]-1
NTDAMAX = time_ad.shape[0]-1

ds_state_phaseI = xr.open_dataset("/scratch2/shreyas/optim_SVC_3point5_50_new_pickup_2/inexact_gn_hessian_cg/state_GNHessCG_iter_25.nc")

log_c_slide_init = ds_state_phaseI["xx_c_slide_init"].data.copy()
log_c_dis_da = np.log10(15659.0)
log_q_geo = ds_state_phaseI["xx_q_geo"].data.copy()
gamma_s = 0.070458*np.ones((JMAX+1, IMAX+1))
log_s_stat = np.log10(5.0)
log_beta1 = np.log10(2.73)
log_beta2 = np.log10(7.28)
log_Pmax = np.log10(0.6)
log_mu = np.log10(9.7155)

data = ds_grl40_bm5_paleo17a_CT4_BH0_m11ka_pkp_init['age_c'].data
log_age_c = np.where(data > 0, np.log10(data), 0)

delta_tda = np.zeros((NTDAMAX+1, JMAX+1, IMAX+1), dtype=float)
for i in range(NTDAMAX+1):
    delta_tda[i, :, :] = delta_ts_init[i]

dict_og_params_fields_vals = {"xx_c_slide_init": log_c_slide_init,
                              "xx_c_dis_da": log_c_dis_da,
                              "xx_q_geo": log_q_geo,
                              "xx_gamma_s": gamma_s,
                              "xx_s_stat": log_s_stat,
                              "xx_beta1": log_beta1,
                              "xx_beta2": log_beta2,
                              "xx_Pmax": log_Pmax,
                              "xx_mu": log_mu,
                              "xx_age_c": log_age_c,
                              "xx_delta_tda": delta_tda}
dict_prior_params_fields_vals = dict_og_params_fields_vals.copy()

dict_params_fields_num_dims = {"xx_c_slide_init": "2D",
                               "xx_c_dis_da": "2D",
                               "xx_q_geo": "2D",
                               "xx_gamma_s": "2D",
                               "xx_s_stat": "2D",
                               "xx_beta1": "2D",
                               "xx_beta2": "2D",
                               "xx_Pmax": "2D",
                               "xx_mu": "2D",
                               "xx_age_c": "3D",
                               "xx_delta_tda": "2DT"}

dict_params_coords = {"time_ad": time_ad,
                      "zeta_c": exp_sigma_level,
                      "zeta_r": zeta_r,
                      "y": yModel40,
                      "x": xModel40}

dict_params_attrs_type = {"xx_c_slide_init": "nodiff",
                          "xx_c_dis_da": "nodiff",
                          "xx_q_geo": "nodiff",
                          "xx_gamma_s": "nodiff",
                          "xx_s_stat": "nodiff",
                          "xx_beta1": "nodiff",
                          "xx_beta2": "nodiff",
                          "xx_Pmax": "nodiff",
                          "xx_mu": "nodiff",
                          "xx_age_c": "nodiff",
                          "xx_delta_tda": "nodiff"}

dict_params_fields_or_scalars = {"xx_c_slide_init": "field",
                                 "xx_c_dis_da": "scalar",
                                 "xx_q_geo": "field",
                                 "xx_gamma_s": "field",
                                 "xx_s_stat": "scalar",
                                 "xx_beta1": "scalar",
                                 "xx_beta2": "scalar",
                                 "xx_Pmax": "scalar",
                                 "xx_mu": "scalar",
                                 "xx_age_c": "field",
                                 "xx_delta_tda": "field"}

year2sec = 3.1556925445e+07
dict_masks_observables = {"H": H_uncert_data**(-2),
                          "age_c": mask_age_c*(age_c_uncert_data*year2sec)**(-2),
                          "V_da_dummy2d": V_uncert_dummy2d_data**(-2)/((IMAX + 1)*(JMAX + 1))}

dict_prior_sigmas = {"xx_c_slide_init": 0.3,
                     "xx_c_dis_da": 0.05,
                     "xx_q_geo": 0.3,
                     "xx_gamma_s": 0.04,
                     "xx_s_stat": 0.04,
                     "xx_beta1": 0.04,
                     "xx_beta2": 0.04,
                     "xx_Pmax": 0.04,
                     "xx_mu": 0.04,
                     "xx_age_c": 0.05,
                     "xx_delta_tda": np.r_[1.0 + np.arange(6)*0.0, 1.67 + np.arange(105)*0.0][::-1]}

dict_prior_gammas = {"xx_c_slide_init": 0.0,
                     "xx_c_dis_da": 0.0,
                     "xx_q_geo": 0.0,
                     "xx_gamma_s": 1.0,
                     "xx_s_stat": 0.0,
                     "xx_beta1": 0.0,
                     "xx_beta2": 0.0,
                     "xx_Pmax": 0.0,
                     "xx_mu": 0.0,
                     "xx_age_c": 0.0,
                     "xx_delta_tda": 1.0}

dict_prior_deltas = {"xx_c_slide_init": 0.0,
                     "xx_c_dis_da": 1.0,
                     "xx_q_geo": 0.0,
                     "xx_gamma_s": 5.e-5,
                     "xx_s_stat": 1.0,
                     "xx_beta1": 1.0,
                     "xx_beta2": 1.0,
                     "xx_Pmax": 1.0,
                     "xx_mu": 1.0,
                     "xx_age_c": 1.0,
                     "xx_delta_tda": 5.e-5}

list_fields_to_ignore = ["xx_c_slide_init", "xx_c_dis_da", "xx_q_geo", "xx_age_c"]

MAX_ITERS_SOR = 100
OMEGA_SOR = 1.5

ds_prior_X = xr.open_dataset("/scratch2/shreyas/optim_AB_nodeltats_noconvinit_constrained_13/prior_X.nc")

DA = optim.DataAssimilation(sicopolis_dir, simulation,
                            dict_sico_out_folder_prefixes, dict_ad_exec_cmds_suffixes,
                            dict_ad_log_file_suffixes, dict_ad_nc_suffixes,
                            dict_og_params_fields_vals, dict_prior_params_fields_vals, dict_params_fields_num_dims, 
                            dict_params_coords, dict_params_attrs_type, dict_params_fields_or_scalars, dict_masks_observables,
                            dict_prior_sigmas, dict_prior_gammas, dict_prior_deltas,
                            MAX_ITERS_SOR, OMEGA_SOR, list_fields_to_ignore, False, None, "/scratch2/shreyas/optim_AB_nodeltats_noconvinit_constrained_prior_stats", None, ds_prior_X, "0006.nc")

for i in range(100):
    ds_subset_sample_prior = DA.sample_prior()
    ds_subset_sample_prior.to_netcdf(f"/scratch2/shreyas/thesis_results/chapter_7/prior_samples/prior_sample_{i+1}.nc")

ds_mean_samples_prior, ds_std_samples_prior = DA.pointwise_marginals("prior", 5000)

ds_mean_samples_prior.to_netcdf(f"/scratch2/shreyas/thesis_results/chapter_7/mean_samples_prior.nc")
ds_std_samples_prior.to_netcdf(f"/scratch2/shreyas/thesis_results/chapter_7/std_samples_prior.nc")